In [19]:
#Importing necessary files
import pandas as pd

In [20]:
""" import os
os.chdir("/Users/larryyyyy/Documents/dse3101")
print(os.getcwd()) """


' import os\nos.chdir("/Users/larryyyyy/Documents/dse3101")\nprint(os.getcwd()) '

In [21]:
bus_202411 = pd.read_csv("dataset/bus stop passenger volum/transport_node_bus_202411.csv")
bus_202412 = pd.read_csv("dataset/bus stop passenger volum/transport_node_bus_202412.csv")
bus_202501 = pd.read_csv("dataset/bus stop passenger volum/transport_node_bus_202501.csv")

bus_stop_volume = pd.concat([bus_202501, bus_202412, bus_202411], axis=0, ignore_index=True)

In [22]:
#Finding nulls
print(bus_stop_volume.isnull().sum())
print(bus_stop_volume.shape)

YEAR_MONTH               0
DAY_TYPE                 0
TIME_PER_HOUR           28
PT_TYPE                  0
PT_CODE                  0
TOTAL_TAP_IN_VOLUME      0
TOTAL_TAP_OUT_VOLUME     0
dtype: int64
(602372, 7)


In [23]:
#Seems like there isnt much data missing except time
null_values = bus_stop_volume[bus_stop_volume.isnull().any(axis=1)]
no_of_null = null_values.groupby('PT_CODE').size()
print(no_of_null)
ptcode = set(null_values['PT_CODE'])

PT_CODE
3239     1
10009    1
10499    1
22009    2
28009    1
28459    1
41009    1
43009    1
44009    2
45009    1
46009    1
47009    1
52009    2
53009    2
54009    1
55509    1
58009    1
64009    1
65009    1
66009    1
75009    1
80009    1
99009    2
dtype: int64


In [24]:
total_data = bus_stop_volume[bus_stop_volume['PT_CODE'].isin(ptcode)].groupby('PT_CODE').size()
print(no_of_null/total_data)


PT_CODE
3239     0.008333
10009    0.008130
10499    0.007874
22009    0.015152
28009    0.007576
28459    0.007812
41009    0.007812
43009    0.007463
44009    0.015152
45009    0.007634
46009    0.007407
47009    0.007874
52009    0.014815
53009    0.015267
54009    0.007634
55509    0.007634
58009    0.007752
64009    0.007692
65009    0.007519
66009    0.007752
75009    0.007299
80009    0.007692
99009    0.015625
dtype: float64


In [25]:
"""Since we can see that the values that are missing only accounts for less than 2% of the data, 
it represents only a small amount and we can safely remove them."""

cleaned_bus_data = bus_stop_volume.dropna()
cleaned_bus_data


,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,2025-01,WEEKDAY,19.0,BUS,75419,185,530
1,2025-01,WEEKDAY,17.0,BUS,46711,1066,133
2,2025-01,WEEKENDS/HOLIDAY,20.0,BUS,43459,93,81
3,2025-01,WEEKENDS/HOLIDAY,22.0,BUS,17069,350,147
4,2025-01,WEEKENDS/HOLIDAY,14.0,BUS,47579,343,203
...,...,...,...,...,...,...,...
602367,2024-11,WEEKDAY,7.0,BUS,46161,148,493
602368,2024-11,WEEKDAY,9.0,BUS,76269,1269,361
602369,2024-11,WEEKDAY,16.0,BUS,66019,646,42
602370,2024-11,WEEKENDS/HOLIDAY,17.0,BUS,24621,33,24


In [26]:
# importing os module for environment variables
import os
# importing necessary functions from dotenv library
from dotenv import load_dotenv
# loading variables from .env file
load_dotenv() 

# accessing and printing value
api_key=os.getenv("API_KEY")

In [27]:
import requests




bus_stops = pd.DataFrame()  # Initialize empty DataFrame

for i in range(0, 5155, 500):
    headers = {
        "AccountKey": api_key,
    }
    

    if i == 0:
        url = "http://datamall2.mytransport.sg/ltaodataservice/BusStops"
    else: 
        url = f"http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip={i}"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        df_new = pd.DataFrame(data['value'])
        
        # Concatenate and reassign
        bus_stops = pd.concat([bus_stops, df_new], axis=0, ignore_index=True)
    else:
        print("Error:", response.status_code, response.text)

# Check the final DataFrame
print(bus_stops.shape)

    

(5155, 5)


In [28]:
bus_stops

,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5150,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5151,99161,Nicoll Dr,Aft Changi Beach CP 3,1.390262,103.992957
5152,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5153,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [29]:
#Creating Geodataframe to join with subzone name
import geopandas as gpd
from shapely.geometry import Point

bus_stop_gdf = gpd.GeoDataFrame(bus_stops,
                             geometry = [Point(x) for x in zip(bus_stops["Longitude"],bus_stops['Latitude'])],
                             crs="EPSG:4326")

bus_stop_gdf

,BusStopCode,RoadName,Description,Latitude,Longitude,geometry
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536,POINT (103.85254 1.29685)
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225,POINT (103.85322 1.29771)
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022,POINT (103.85302 1.29699)
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414,POINT (103.85441 1.29667)
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491,POINT (103.85549 1.29821)
...,...,...,...,...,...,...
5150,99139,Changi Village Rd,Blk 5,1.388195,103.987234,POINT (103.98723 1.38819)
5151,99161,Nicoll Dr,Aft Changi Beach CP 3,1.390262,103.992957,POINT (103.99296 1.39026)
5152,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021,POINT (103.99102 1.39113)
5153,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503,POINT (103.9885 1.38775)


In [30]:
#Importing mapping area
polygon = gpd.read_file("dataset/cleaned_mapping_data.gpkg")
polygon

,Name,Description,subzone,planning_area,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,DEPOT ROAD,BUKIT MERAH,"MULTIPOLYGON Z (((103.81454 1.28239 0, 103.817..."
1,kml_2,<center><table><tr><th colspan='2' align='cent...,BUKIT MERAH,BUKIT MERAH,"MULTIPOLYGON Z (((103.82209 1.28049 0, 103.822..."
2,kml_3,<center><table><tr><th colspan='2' align='cent...,CHINATOWN,OUTRAM,"MULTIPOLYGON Z (((103.84375 1.28508 0, 103.844..."
3,kml_4,<center><table><tr><th colspan='2' align='cent...,PHILLIP,DOWNTOWN CORE,"MULTIPOLYGON Z (((103.84962 1.28412 0, 103.849..."
4,kml_5,<center><table><tr><th colspan='2' align='cent...,RAFFLES PLACE,DOWNTOWN CORE,"MULTIPOLYGON Z (((103.85253 1.28617 0, 103.852..."
...,...,...,...,...,...
327,kml_328,<center><table><tr><th colspan='2' align='cent...,UPPER THOMSON,BISHAN,"MULTIPOLYGON Z (((103.84263 1.36323 0, 103.842..."
328,kml_329,<center><table><tr><th colspan='2' align='cent...,SHANGRI-LA,ANG MO KIO,"MULTIPOLYGON Z (((103.83537 1.37093 0, 103.835..."
329,kml_330,<center><table><tr><th colspan='2' align='cent...,TOWNSVILLE,ANG MO KIO,"MULTIPOLYGON Z (((103.84814 1.36879 0, 103.848..."
330,kml_331,<center><table><tr><th colspan='2' align='cent...,MARYMOUNT,BISHAN,"MULTIPOLYGON Z (((103.84652 1.34521 0, 103.846..."


In [13]:
#Displaying bus stops with their respective zones
bus_stops_with_zones = gpd.sjoin(bus_stop_gdf, polygon, predicate="within", how="left")
bus_stops_with_zones =bus_stops_with_zones[["BusStopCode","subzone","planning_area"]]
bus_stops_with_zones['BusStopCode']= bus_stops_with_zones['BusStopCode'].astype(int)
bus_stops_with_zones
#non_exist = set(bus_stops_with_zones[bus_stops_with_zones.isnull().any(axis=1)]['BusStopCode'])
#non_exist


,BusStopCode,subzone,planning_area
0,1012,VICTORIA,ROCHOR
1,1013,VICTORIA,ROCHOR
2,1019,BUGIS,DOWNTOWN CORE
3,1029,BUGIS,DOWNTOWN CORE
4,1039,BUGIS,DOWNTOWN CORE
...,...,...,...
5150,99139,CHANGI POINT,CHANGI
5151,99161,CHANGI AIRPORT,CHANGI
5152,99171,CHANGI AIRPORT,CHANGI
5153,99181,CHANGI POINT,CHANGI


In [14]:
overall_bus_stop_df = cleaned_bus_data.merge(bus_stops_with_zones, left_on="PT_CODE", right_on="BusStopCode", how='left')
overall_bus_stop_df[overall_bus_stop_df['subzone'].isnull()].sort_values(by='TOTAL_TAP_IN_VOLUME')


,YEAR_MONTH,DAY_TYPE,TIME_PER_HOUR,PT_TYPE,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME,BusStopCode,subzone,planning_area
216411,2024-12,WEEKENDS/HOLIDAY,0.0,BUS,13139,0,4,NaN,NaN,NaN
157796,2025-01,WEEKENDS/HOLIDAY,7.0,BUS,47711,0,48,47711.0,NaN,NaN
500723,2024-11,WEEKENDS/HOLIDAY,1.0,BUS,46239,0,80,46239.0,NaN,NaN
563574,2024-11,WEEKENDS/HOLIDAY,5.0,BUS,51023,0,2,NaN,NaN,NaN
318960,2024-12,WEEKDAY,8.0,BUS,51023,0,91,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
339950,2024-12,WEEKDAY,7.0,BUS,46219,23811,1273,46219.0,NaN,NaN
491213,2024-11,WEEKDAY,7.0,BUS,46219,24342,1033,46219.0,NaN,NaN
65346,2025-01,WEEKDAY,6.0,BUS,46219,31481,1171,46219.0,NaN,NaN
356275,2024-12,WEEKDAY,6.0,BUS,46219,32693,1240,46219.0,NaN,NaN


In [15]:
#Some of the PTCODE does not exist in the API Call and it represents a small subset of our data, thus for simplicity we drop them
overall_bus_stop_df = overall_bus_stop_df.dropna()
overall_bus_stop_df.shape

(601505, 10)

In [38]:
agg_df = overall_bus_stop_df.groupby(by=['planning_area','subzone','YEAR_MONTH','DAY_TYPE'])[['TOTAL_TAP_IN_VOLUME','TOTAL_TAP_OUT_VOLUME']].sum().reset_index()
agg_df




,planning_area,subzone,YEAR_MONTH,DAY_TYPE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME
0,ANG MO KIO,ANG MO KIO TOWN CENTRE,2024-11,WEEKDAY,647724,614159
1,ANG MO KIO,ANG MO KIO TOWN CENTRE,2024-11,WEEKENDS/HOLIDAY,245244,237074
2,ANG MO KIO,ANG MO KIO TOWN CENTRE,2024-12,WEEKDAY,639998,605007
3,ANG MO KIO,ANG MO KIO TOWN CENTRE,2024-12,WEEKENDS/HOLIDAY,268806,261155
4,ANG MO KIO,ANG MO KIO TOWN CENTRE,2025-01,WEEKDAY,673397,639398
...,...,...,...,...,...,...
1879,YISHUN,YISHUN WEST,2024-11,WEEKENDS/HOLIDAY,313436,284243
1880,YISHUN,YISHUN WEST,2024-12,WEEKDAY,793807,705494
1881,YISHUN,YISHUN WEST,2024-12,WEEKENDS/HOLIDAY,342196,308295
1882,YISHUN,YISHUN WEST,2025-01,WEEKDAY,887000,791772
